In [97]:
import pandas as pd
pd.set_option('display.max_columns', None)
import numpy as np
from IPython.core.display import display, HTML
import matplotlib.pyplot as plt
import pickle
from collections import defaultdict

In [2]:
# natural language processing
import nltk
import re
import string
# nltk.download('wordnet')
# nltk.download('omw-1.4')
from nltk.corpus import stopwords
from nltk.stem.lancaster import LancasterStemmer
from nltk.tokenize import RegexpTokenizer
from nltk.stem import WordNetLemmatizer
from nltk.tag import pos_tag
from nltk.chunk import ne_chunk
from sklearn.feature_extraction.text import CountVectorizer
from nltk.tokenize import word_tokenize
from sklearn.feature_extraction.text import TfidfVectorizer

In [176]:
# load dataset
prod = ['engine-oil-recommender','tuning-remap',
        'tyre-recommender','wiper-blades']
df = pd.read_csv('COMPLETE_car-batteries.csv')
for product in prod:
    df = df.append(pd.read_csv(f'COMPLETE_{product}.csv'), ignore_index=True)

df = df.drop(['Unnamed: 0.1', 'Unnamed: 0','Unnamed: 0.2'], axis=1)
df

C:\Users\farea\AppData\Local\Temp\ipykernel_6444\1985822615.py:6: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(pd.read_csv(f'COMPLETE_{product}.csv'), ignore_index=True)
C:\Users\farea\AppData\Local\Temp\ipykernel_6444\1985822615.py:6: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(pd.read_csv(f'COMPLETE_{product}.csv'), ignore_index=True)
C:\Users\farea\AppData\Local\Temp\ipykernel_6444\1985822615.py:6: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(pd.read_csv(f'COMPLETE_{product}.csv'), ignore_index=True)
C:\Users\farea\AppData\Local\Temp\ipykernel_6444\1985822615.py:6: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use

,id,title,vendor,category,prod_type,publish,price (MYR),compare_at_price (MYR),weight (grams),available,sku,rating,review_count,questions,review
0,7.774600e+12,Lucas LSLA4-6 SLA Battery 6V 4Ah,Lucas,Car Batteries,Battery,2022-07-15T08:11:29+01:00,25,25.0,750,True,LUCAS49,5.000000,1736.0,6.0,['Fitted this to Peugeot 207 1.6 Diesel FN11 y...
1,7.774600e+12,Lucas LSLA20-12 SLA Battery 12V 20Ah,Lucas,Car Batteries,Battery,2022-07-15T08:11:25+01:00,176,165.0,7000,True,LUCAS47,5.000000,1844.0,4.0,"['Well packed and delivered on time.', 'Arrive..."
2,7.770420e+12,Exide AGM EK131 Auxiliary Battery,Exide,Car Batteries,Battery,2022-07-12T02:44:57+01:00,219,220.0,5000,True,444779370,4.997804,1366.0,7.0,['My package got a problem from DHL and I cont...
3,8.007850e+12,Xtreme Racing Series XR-600 AGM Battery 26Ah 600A,XTREME,Car Batteries,Battery,2023-02-13T04:49:43+00:00,264,299.0,4700,True,XTREME-XR600,4.822430,321.0,0.0,['Really impressed with the quality of this pr...
4,8.026230e+12,Varta H16 Trailer battery - Black Promotive Co...,Varta,Car Batteries,Battery,2025-02-04T06:09:46+00:00,588,588.0,24400,True,Varta-H16,0.000000,0.0,0.0,[]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
67357,7.790320e+12,Bosch Super Plus Universal Wiper Blade Set Sp2...,Bosch,Wiper Blades,Wiper Blade,2022-07-26T06:46:42+01:00,55,55.0,5000,True,485775610,0.000000,0.0,0.0,[]
67358,7.790320e+12,Bosch Super Plus Wiper Blade Rear H312,Bosch,Wiper Blades,Wiper Blade,2022-07-26T06:46:33+01:00,25,25.0,5000,True,485773810,0.000000,0.0,0.0,[]
67359,7.790320e+12,Bosch Super Plus Universal Wiper Blade Set Sp2...,Bosch,Wiper Blades,Wiper Blade,2022-07-26T06:46:27+01:00,47,47.0,5000,True,485775540,0.000000,0.0,0.0,[]
67360,7.790320e+12,Bosch Aerotwin Flat Wiper Blade Set A962S Front,Bosch,Wiper Blades,Wiper Blade,2022-07-26T06:46:23+01:00,119,119.0,5000,True,485775030,0.000000,0.0,0.0,[]


In [126]:
# Clean special character from reviews
df["FullReview"] = ["".join(re.sub(r"\[\'|\]$|(^\s\')|\\r|(^\s\")|\!|[*?\'$]|(,\s\")|(\"\,)|\[","", review)) for review in df.review]
ftext = []
for rev in range(len(df.FullReview)):
    if (df.FullReview[rev]):
        ftext += df.FullReview[rev].split('., ')
        
len(ftext)

48729

In [5]:
from nltk.corpus import stopwords

In [64]:
#create CountVectorizer not counting rare words/misspellings
sw = stopwords.words('english')
vectorizer = CountVectorizer(min_df=10, stop_words=sw)
doc_word = vectorizer.fit_transform(df.FullReview.str.replace(r'\d+','', regex=True))
doc_word

#create TF-IDF matrix for full corpus on each row
tfidfvectorizer = TfidfVectorizer(min_df=10, stop_words=sw)
tfidf_corpus = tfidfvectorizer.fit_transform(df.FullReview.str.replace(r'\d+','',regex=True))
tfidf_corpus

<3783x1583 sparse matrix of type '<class 'numpy.float64'>'
	with 90223 stored elements in Compressed Sparse Row format>

In [65]:
pd.DataFrame(vectorizer.transform(['ML Performance is great and fast delivery']).toarray()).sum(axis=1)

0    5
dtype: int64

In [8]:
# Checking if CountVectorizer return a correct output
pd.DataFrame(vectorizer.transform([df.FullReview[0]]).toarray()).sum(axis=1)[0],len(df.FullReview[0])

(47, 918)

In [137]:
sw = stopwords.words('english') + ['able','absolutely','actually','ad','africa','againfirst','againi','againive','againml','againml','againmlps','againthe','ago','al',
                                   'alaways','alot','alwaysml','alwaysthe','america','apart','aswell','australia','azren','banana','basildon','belated','bmwparts','brainer','braught',
                                   'call','cannott','cant','car','cars','cel','communicative','competitively','couldnt','dear','deffinately','deffinatley','definately','degs',
                                   'deliveryml','deliverymlps','deliverys','deliverythe','delivring','diappoint','didnt','doesnt','dom','dont','dontt','enquirey','equireies','espect',
                                   'ethan','expensivethere','fab','fing','france','futureive','futureml','futurethe','gals','german','guysi','guy','guysive','guysml','guysmlps','guysthe',
                                   'h','haha','happyml','happythe','heaverier','hi','hesitiate','ho','holland','however','ie','im','jb','jonathan','klarna','korea','lads','lee',
                                   'malaysia','manyparts','marques','marshall','marshallthe','mauritius','mi','mika','mishimoto','ml','mli','mlim','mlive','mlml','mlmlps','mlp',
                                   'mlperformance','mlperformace','mlperformances','mlpml','mlpmlps','mlpreformance','mlps','mlpthe','mls','mlthe','monday','moneyt','mor','moreml','moremlps',
                                   'morethe','mot','n','nagain','nall','nalso','nand','nas','nbest','nbravo','nbusiness','ncheers','ncommunication','ncongrats','ncouldnt','ncustomer',
                                   'ndefinitely','ndelivery','nease','neednt','neil','neither','nervy','nexcellent','nfantastic','nfrom','nget','ngood','ngraham','ngreat','nhead','nhave',
                                   'nhesitate','nhighly','nhonestly','ni','nicholas','nid','nit','nits','nive','nkeep','nlook','nlooking','nmany','nml','nmuch','nmy','nnext','nno','nnote',
                                   'also','bmws','even','friday','saturday','get','hesitiate','ive','like','log','keep','mlps','ml','note','performanceive','please','reflash',
                                   'sam','say','thank','tingreat','wednesday','welcome','would','tin','e','f','oh','ok','packagedbox','per','perf','portugal']

In [138]:
# Stemming and lemmatizing
from nltk.stem.lancaster import LancasterStemmer
from nltk.tokenize import RegexpTokenizer
from nltk.stem import WordNetLemmatizer

def preprocess_textstem(text):
    # Remove numbers
    text = re.sub(r'\d+', '', text)
    
    # Tokenise words while ignoring punctuation
    tokeniser = RegexpTokenizer(r'\w+')
    tokens = tokeniser.tokenize(text)
    
     # Lowercase and stem
    stemmer = LancasterStemmer()
    stems = [stemmer.stem(token.lower()) for token in tokens]
    
    # Remove stopwords
    keywords= [stem for stem in stems if stem not in sw]
    return keywords

def preprocess_textlemma(text):
    # Remove numbers
    text = re.sub(r'\d+', '', text)
    
    # Tokenise words while ignoring punctuation
    tokeniser = RegexpTokenizer(r'\w+')
    tokens = tokeniser.tokenize(text)
    
    # Lowercase and lemmatise 
    lemmatiser = WordNetLemmatizer()
    lemmas = [lemmatiser.lemmatize(token.lower(), pos='v') for token in tokens]
    
    # Remove stopwords
    keywords= [lemma for lemma in lemmas if lemma not in sw]
    return keywords

In [10]:
# Create instance for TF-IDF vectorizer
sample_vector = TfidfVectorizer(analyzer=preprocess_textlemma)
sample_lemma = sample_vector.fit_transform(df.FullReview.head())
# Convert to sparse matrix
sample_lemma = pd.DataFrame.sparse.from_spmatrix(sample_lemma)
# Save mapping on which index refers to which words
col_map = {v:k for k, v in sample_vector.vocabulary_.items()}
# Rename each column using the mapping
for col in sample_lemma.columns:
    sample_lemma.rename(columns={col:col_map[col]}, inplace=True)
sample_lemma

,absolute,actually,adapter,aesthetics,also,amaze,apologise,app,arrive,beautiful,big,birthday,bite,bmw,bmws,braught,breeze,brilliant,car,change,class,cod,come,communication,company,complain,complete,confirm,connection,contact,customer,damage,definitely,delivery,describe,dispatch,downtime,e,easy,economy,ensure,even,every,everything,exactly,expect,f,fast,fault,feel,focus,free,friday,fuel,future,g,get,good,goodies,great,hand,happy,hesitiate,high,husband,improve,improvement,instal,instantly,intend,issue,ive,job,keep,lads,later,like,log,longer,look,make,mega,mention,mhd,minimal,ml,mlps,modern,multiple,need,nice,norm,note,notice,offer,okay,order,original,overall,package,part,penny,performance,performanceive,phone,please,present,product,prompt,provide,purchase,quality,quick,read,really,rebook,reflash,refresh,reliable,repair,reply,reset,response,reviewers,run,sam,say,second,seem,series,service,ship,side,significant,simple,sleek,slow,sound,spot,superb,sync,take,thank,throughout,time,tin,tingreat,tool,touch,transform,tune,udc,unexpected,universal,update,use,vehicle,want,wednesday,welcome,well,work,worth,would,write
0,0.000000,0.000000,0.082173,0.000000,0.000000,0.000000,0.082173,0.082173,0.082173,0.000000,0.000000,0.000000,0.000000,0.082173,0.082173,0.000000,0.082173,0.000000,0.066297,0.000000,0.000000,0.24652,0.000000,0.000000,0.000000,0.000000,0.000000,0.082173,0.164347,0.082173,0.000000,0.164347,0.000000,0.055032,0.000000,0.000000,0.000000,0.082173,0.066297,0.000000,0.000000,0.082173,0.000000,0.164347,0.066297,0.164347,0.082173,0.220130,0.082173,0.000000,0.000000,0.082173,0.000000,0.000000,0.000000,0.082173,0.082173,0.066297,0.082173,0.132594,0.000000,0.164347,0.000000,0.082173,0.000000,0.000000,0.000000,0.000000,0.000000,0.082173,0.000000,0.000000,0.082173,0.000000,0.000000,0.082173,0.082173,0.082173,0.082173,0.000000,0.082173,0.082173,0.000000,0.082173,0.000000,0.132594,0.000000,0.000000,0.000000,0.082173,0.000000,0.000000,0.000000,0.000000,0.000000,0.082173,0.000000,0.082173,0.000000,0.066297,0.066297,0.000000,0.198891,0.000000,0.000000,0.000000,0.000000,0.198891,0.000000,0.000000,0.000000,0.000000,0.066297,0.082173,0.000000,0.082173,0.082173,0.000000,0.000000,0.000000,0.082173,0.082173,0.082173,0.000000,0.082173,0.000000,0.164347,0.000000,0.082173,0.082173,0.000000,0.164347,0.000000,0.000000,0.000000,0.000000,0.082173,0.000000,0.000000,0.000000,0.000000,0.082173,0.000000,0.000000,0.000000,0.082173,0.082173,0.082173,0.066297,0.082173,0.082173,0.000000,0.000000,0.082173,0.066297,0.165097,0.000000,0.000000,0.000000,0.000000,0.164347,0.410867,0.000000,0.066297,0.000000
1,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000

Okay to use full review corpus

In [139]:
# Create instance for TF-IDF vectorizer
tfidfvectorizer = TfidfVectorizer(analyzer=preprocess_textlemma,
                                  min_df=20)
lemma = tfidfvectorizer.fit(ftext)
lemma = tfidfvectorizer.transform(df.FullReview)
# Convert to sparse matrix
lemma = pd.DataFrame.sparse.from_spmatrix(lemma)
# Save mapping on which index refers to which words
col_map = {v:k for k, v in tfidfvectorizer.vocabulary_.items()}
# Rename each column using the mapping
for col in lemma.columns:
    lemma.rename(columns={col:col_map[col]}, inplace=True)
lemma

absolute  acceleration  accompany  achieve  across  add  addition  \
0           0.0           0.0   0.000000      0.0     0.0  0.0       0.0   
1           0.0           0.0   0.000000      0.0     0.0  0.0       0.0   
2           0.0           0.0   0.076807      0.0     0.0  0.0       0.0   
3           0.0           0.0   0.000000      0.0     0.0  0.0       0.0   
4           0.0           0.0   0.000000      0.0     0.0  0.0       0.0   
...         ...           ...        ...      ...     ...  ...       ...   
67357       0.0           0.0   0.000000      0.0     0.0  0.0       0.0   
67358       0.0           0.0   0.000000      0.0     0.0  0.0       0.0   
67359       0.0           0.0   0.000000      0.0     0.0  0.0       0.0   
67360       0.0           0.0   0.000000      0.0     0.0  0.0       0.0   
67361       0.0           0.0   0.000000      0.0     0.0  0.0       0.0   

       additional  address  admit  advancements  advantage  advertise  \
0             0.0      0.0    0.0           0.0        0.0        0.0   
1             0.0      0.0    0.0           0.0        0.0        0.0   
2             0.0      0.0    0.0           0.0        0.0        0.0   
3             0.0      0.0    0.0           0.0        0.0        0.0   
4             0.0      0.0    0.0           0.0        0.0        0.0   
...           ...      ...    ...           ...        ...        ...   
67357         0.0      0.0    0.0           0.0        0.0        0.0   
67358         0.0      0.0    0.0           0.0        0.0        0.0   
67359         0.0      0.0    0.0           0.0        0.0        0.0   
67360         0.0      0.0    0.0           0.0        0.0        0.0   
67361         0.0      0.0    0.0           0.0        0.0        0.0   

         advice  advise  aesthetics  afe  affordability  affordable  afterall  \
0      0.000000     0.0         0.0  0.0            0.0         0.0       0.0   
1      0.000000     0.0         0.0  0.0            0.0         0.0       0.0   
2      0.051825     0.0         0.0  0.0            0.0         0.0       0.0   
3      0.000000     0.0         0.0  0.0            0.0         0.0       0.0   
4      0.000000     0.0         0.0  0.0            0.0         0.0       0.0   
...         ...     ...         ...  ...            ...         ...       ...   
67357  0.000000     0.0         0.0  0.0            0.0         0.0       0.0   
67358  0.000000     0.0         0.0  0.0            0.0         0.0       0.0   
67359  0.000000     0.0         0.0  0.0            0.0         0.0       0.0   
67360  0.000000     0.0         0.0  0.0            0.0         0.0       0.0   
67361  0.000000     0.0         0.0  0.0            0.0         0.0       0.0   

       aftermarket  afternoon  air  alike  allow  almost  alone  along  \
0              0.0   0.000000  0.0    0.0    0.0     0.0    0.0    0.0   
1              0.0   0.000000  0.0    0.0    0.0     0.0    0.0    0.0   
2              0.0   0.078895  0.0    0.0    0.0     0.0    0.0    0.0   
3              0.0   0.000000  0.0    0.0    0.0     0.0    0.0    0.0   
4              0.0   0.000000  0.0    0.0    0.0     0.0    0.0    0.0   
...            ...        ...  ...    ...    ...     ...    ...    ...   
67357          0.0   0.000000  0.0    0.0    0.0     0.0    0.0    0.0   
67358          0.0   0.000000  0.0    0.0    0.0     0.0    0.0    0.0   
67359          0.0   0.000000  0.0    0.0    0.0     0.0    0.0    0.0   
67360          0.0   0.000000  0.0    0.0    0.0     0.0    0.0    0.0   
67361          0.0   0.000000  0.0    0.0    0.0     0.0    0.0    0.0   

       already  alternative  although    always     amaze    amazon  amend  \
0          0.0     0.070064       0.0  0.111889  0.000000  0.091962    0.0   
1          0.0     0.000000       0.0  0.110623  0.000000  0.090921    0.0   
2          0.0     0.000000       0.0  0.029478  0.000000  0.000000    0.0   
3          0.0     0.000000       0.0  0.09

In [140]:
with open('LEMMA_mlperformance.pickle','wb') as to_write:
    pickle.dump(lemma, to_write)

In [134]:
df.shape

(67362, 15)

# Non-negative Matrix Factorization
* For dimensionality reduction

In [19]:
# import visualization packages
import matplotlib.patches as mpatches
import matplotlib
%matplotlib inline

from nltk.tokenize import word_tokenize

In [135]:
# Function used to display topics from NMF models
def display_topics(model, feature_names, no_top_words, topic_names=None):
    for ix, topic in enumerate(model.components_):
        if not topic_names or not topic_names[ix]:
            print("\nTopic ", ix)
        else:
            print("\nTopic: '",topic_names[ix],"'")
        print(", ".join([feature_names[i] for i in topic.argsort()[:-no_top_words - 1:-1]]))

In [14]:
# Potential topics for review
from sklearn.decomposition import NMF

In [150]:
topic_num = 7
nmf_tfidf = NMF(topic_num)
topic_tfidf = nmf_tfidf.fit_transform(lemma)
filter_topic_tfidf = pd.DataFrame(nmf_tfidf.components_.round(4),
                                  index = [f'component_{num}' for num in range(1,topic_num+1)],
                                  columns=tfidfvectorizer.get_feature_names_out())
display_topics(nmf_tfidf, tfidfvectorizer.get_feature_names_out(), 20)


Topic  0
use, performance, recommend, buy, purchase, time, highly, definitely, first, communication, deliver, future, happy, never, products, items, best, helpful, company, package

Topic  1
part, performances, vehicle, improve, make, overall, vehicles, drive, experience, difference, prove, enhance, feel, durability, offer, impress, save, reliable, road, quality

Topic  2
order, receive, day, next, performance, arrive, within, item, time, place, deliver, days, staff, package, definitely, make, complain, email, first, minutes

Topic  3
great, service, product, item, price, team, usual, make, many, fault, well, quality, fit, overall, everything, definitely, amaze, every, performance, issue

Topic  4
good, quality, work, price, really, product, job, deal, team, care, deliver, must, time, question, cheapest, touch, expect, quickly, well, soon

Topic  5
service, always, customer, excellent, performance, provide, team, top, go, happy, product, notch, support, fantastic, back, usual, satisfa

In [149]:
filter_topic_tfidf.sort_values(by='component_8', axis=1, ascending=False)

always    team     top  service  performance   notch    come  \
component_1  0.0000  0.0000  0.0377   0.0725       2.1742  0.0000  0.2545   
component_2  0.0000  0.0000  0.0763   0.0000       0.3202  0.0611  0.0436   
component_3  0.0000  0.0064  0.0000   0.0000       0.6170  0.0000  0.0986   
component_4  0.0000  0.5266  0.0000   2.0034       0.1830  0.0000  0.0000   
component_5  0.0000  0.3113  0.1885   0.0167       0.0000  0.0000  0.0354   
component_6  0.0000  0.0000  0.1495   2.0073       0.9357  0.1232  0.0000   
component_7  0.0000  0.0000  0.0049   0.0707       0.2888  0.0156  0.0000   
component_8  3.5014  1.1114  0.8164   0.5783       0.4893  0.4885  0.4718   

             fantastic    help    back      go   quick     way    time  \
component_1     0.0000  0.0630  0.1742  0.1472  0.2610  0.0525  1.0714   
component_2     0.0000  0.2215  0.0500  0.0411  0.0117  0.0192  0.1581   
component_3     0.0785  0.0000  0.1497  0.0129  0.0000  0.0368  0.4145   
component_4     0.1927  0.1083  0.0000  0.0000  0.0000  0.0539  0.0000   
component_5     0.0077  0.0235  0.1283  0.0000  0.0239  0.0096  0.2269   
component_6     0.0865  0.0332  0.1326  0.3724  0.0467  0.0558  0.0000   
component_7     0.3110  0.0000  0.0000  0.1412  0.8689  0.0905  0.0000   
component_8     0.4659  0.4315  0.3973  0.3708  0.2917  0.2860  0.2801   

             advise  dispatch  deliver    best   every  product  inform  \
component_1  0.0000    0.0000   0.4098  0.3426  0.0605   0.3151  0.1306   
component_2  0.0000    0.0000   0.1486  0.0213  0.0309   0.0000  0.0323   
component_3  0.0000    0.0076   0.3776  0.2106  0.2111   0.0504  0.1244   
component_4  0.0000    0.0000   0.0000  0.0663  0.1976   1.0469  0.0495   
component_5  0.0190    0.0000   0.2323  0.0779  0.0000   0.5713  0.0000   
component_6  0.0000    0.0236   0.0000  0.0000  0.0000   0.4423  0.0000   
component_7  0.0000    0.2323   0.0000  0.0343  0.0798   0.2332  0.0000   
component_8  0.2703    0.2684   0.2680  0.2611  0.2287   0.2286  0.2229   

               fail   issue    well  flawless    turn  enough     one    many  \
component_1  0.0739  0.2185  0.2768    0.0000  0.0000  0.0640  0.1460  0.0982   
component_2  0.0233  0.1960  0.0333    0.0000  0.0035  0.0000  0.1049  0.0183   
component_3  0.0000  0.0000  0.1640    0.0000  0.1442  0.0535  0.1590  0.0000   
component_4  0.0000  0.1825  0.2876    0.0000  0.0000  0.0000  0.0142  0.3012   
component_5  0.0000  0.0000  0.1995    0.0000  0.0222  0.0000  0.1569  0.0279   
component_6  0.0000  0.0265  0.0709    0.0207  0.0000  0.0024  0.0031  0.0000   
component_7  0.0000  0.0091  0.0714    0.0000  0.0000  0.0000  0.0000  0.0000   
component_8  0.2203  0.2185  0.2170    0.2123  0.1993  0.1885  0.1821  0.1777   

              happy   items  continue  people  around   super  helpful  \
component_1  0.4525  0.3732    0.1627  0.1394  0.0644  0.0000   0.3541   
component_2  0.0000  0.0048    0.0000  0.0112  0.0000  0.0000   0.0112   
component_3  0.0022  0.0643    0.0000  0.0000  0.0000  0.0000   0.0580   
component_4  0.0000  0.0000    0.0168  0.0491  0.0299  0.0000   0.0000   
component_5  0.0275  0.0400    0.0000  0.0302  0.0000  0.0000   0.0000   
component_6  0.5347  0.0000    0.0567  0.0000  0.0000  0.0000   0.1034   
component_7  0.0000  0.0000    0.0189  0.0119  0.0000  0.8854   0.4183   
component_8  0.1760  0.1748    0.1634  0.1567  0.1540  0.1532   0.1530   

             quality  reason  professional   arise   worry     day    next  \
component_1   0.0000  0.0000        0.0000  0.0000  0.0492  0.0000  0.0000   
component_2   0.3734  0.0000        0.0167  0.0000  0.0067  0.0000  0.0000   
component_3   0.0000  0.0000        0.0000  0.0000  0.0000  1.0651  0.9704   
component_4   0.2559  0.0000        0.0214  0.0000  0.0000  0.0000  0.0000   
component_5   1.4012  0.0000        0.0000  0.0000  0.0000  0.0853  0.0615   
component_6   0.0000  0.0000        0.0126  0.0000  0.0181  0.0000  0.0000   
component_7   0.0000  0.0000     

In [151]:
topic_tfidf = pd.DataFrame(topic_tfidf, columns=[
 'recommended_purchase','performance_vehicle_improve','delivery_time',
 'great_service_product_item','good_quality','excellent_customer_service',
 'fast_delivery'])
topic_tfidf.shape

(67362, 7)

In [ ]:
complete_data = pd.concat([df, topic_tfidf], axis=1)
with open('COMPLETE_topic_breakdown.pickle', 'wb') as to_write:
    pickle.dump(complete_data, to_write)

In [180]:
piedf = complete_data.groupby(['category','prod_type']).size().reset_index(name='count')
piedf['percentage']=piedf['count']/sum(piedf['count']) * 100.0
with open('piedf.pickle','wb') as to_write:
    pickle.dump(piedf, to_write)